In [1]:
## import needed libraries
import pandas as pd #Dataframes
import numpy as np #Numerics
import json #JSON
import ast #Eval datatype
import gzip ##
from textblob import TextBlob## NPL
import re
import unicodedata
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
## read datasets
df_sg=pd.read_csv("Dataset/Cleaned_Dataset/steam_games.csv")
df_ui=pd.read_csv("Dataset/Cleaned_Dataset/user_items.csv")
df_ur=pd.read_csv("Dataset/Cleaned_Dataset/user_reviews.csv")
df_sent=pd.read_csv("Dataset/Cleaned_Dataset/user_reviews_sent.csv")

In [3]:
df_sent## dataset 

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,year_posted,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,Simple yet with great replayability In my opin...,2011.0,1
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,Its unique and worth a playthrough,2011.0,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,Great atmosphere The gunplay can be a bit chun...,2011.0,1
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014.0,1
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,For a simple its actually not all that simple ...,2013.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",NaN,Posted July 10.,NaN,70.0,No ratings yet,True,a must have classic from steam definitely wort...,NaN,2
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",NaN,Posted July 8.,NaN,362890.0,No ratings yet,True,this game is a perfect remake of the original ...,NaN,1
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,Posted July 3.,NaN,273110.0,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN,1
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",NaN,Posted July 20.,NaN,730.0,No ratings yet,True,D,NaN,1


In [4]:
df_sg## dataset

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro,2018.0
1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro,2018.0
2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro,2018.0
3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro,2018.0
4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85623,NaN,Adventure,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,SinFecha,"['Early Access', 'Adventure', 'Indie', 'Action...",http://steamcommunity.com/app/681550/reviews/?...,"['Single-player', 'Stats', 'Steam Leaderboards...",4.99,True,681550,NaN,NaN
85624,NaN,Indie,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,SinFecha,"['Early Access', 'Adventure', 'Indie', 'Action...",http://steamcommunity.com/app/681550/reviews/?...,"['Single-player', 'Stats', 'Steam Leaderboards...",4.99,True,681550,NaN,NaN
85625,NaN,Action,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,SinFecha,"['Early Access', 'Adventure', 'Indie', 'Action...",http://steamcommunity.com/app/681550/reviews/?...,"['Single-player', 'Stats', 'Steam Leaderboards...",4.99,True,681550,NaN,NaN
85626,NaN,Simulation,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,SinFecha,"['Early Access', 'Adventure', 'Indie', 'Action...",http://steamcommunity.com/app/681550/reviews/?...,"['Single-player', 'Stats', 'Steam Leaderboards...",4.99,True,681550,NaN,NaN


In [5]:
df_ml1=df_sg[["tags","app_name","id","specs"]]## Extract useful columns

In [6]:
## Drop duplicated rows
df_ml1.drop_duplicates(inplace=True)
df_ml1.reset_index(drop=True,inplace=True)

C:\Users\angeo\AppData\Local\Temp\ipykernel_21612\3394998699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml1.drop_duplicates(inplace=True)


In [7]:
df_ml1 ## view dataset

,tags,app_name,id,specs
0,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",Lost Summoner Kitty,761140,['Single-player']
1,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",Ironbound,643980,"['Single-player', 'Multi-player', 'Online Mult..."
2,"['Free to Play', 'Simulation', 'Sports', 'Casu...",Real Pool 3D - Poolians,670290,"['Single-player', 'Multi-player', 'Online Mult..."
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,767400,['Single-player']
4,"['Action', 'Indie', 'Casual', 'Sports']",Log Challenge,773570,"['Single-player', 'Full controller support', '..."
...,...,...,...,...
32127,"['Strategy', 'Indie', 'Casual', 'Simulation']",Colony On Mars,773640,"['Single-player', 'Steam Achievements']"
32128,"['Strategy', 'Indie', 'Casual']",LOGistICAL: South Africa,733530,"['Single-player', 'Steam Achievements', 'Steam..."
32129,"['Indie', 'Simulation', 'Racing']",Russian Roads,610660,"['Single-player', 'Steam Achievements', 'Steam..."
32130,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",EXIT 2 - Directions,658870,"['Single-player', 'Steam Achievements', 'Steam..."


In [8]:
df_ml2=df_sent[["item_id","recommend","sentiment_analysis"]][(df_sent["recommend"]==True) & (df_sent["sentiment_analysis"].isin([1,2]))] ## Due to computational costs, the most recommended games are used to reduce database

In [9]:
df_ml2.drop_duplicates(subset="item_id",inplace=True)## Drop duplicate games rows
df_ml2.reset_index(drop=True,inplace=True)

In [26]:
df_joinml1 = pd.merge(df_ml2, df_ml1, left_on='item_id', right_on='id', how='inner')## join datasets 

In [27]:
df_joinml1["specs"][df_joinml1["specs"].isnull()]="[]"## replace NaN values in column specs to list structure

C:\Users\angeo\AppData\Local\Temp\ipykernel_21612\901597927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joinml1["specs"][df_joinml1["specs"].isnull()]="[]"## replace NaN values in column specs to list structure


In [28]:
df_joinml1

,item_id,recommend,sentiment_analysis,tags,app_name,id,specs
0,1250.0,True,1,"['FPS', 'Zombies', 'Co-op', 'Survival', 'Actio...",Killing Floor,1250,"['Single-player', 'Multi-player', 'Co-op', 'Cr..."
1,22200.0,True,2,"['Action', 'Indie', 'Surreal', ""Beat 'em up"", ...",Zeno Clash,22200,"['Single-player', 'Steam Achievements', 'Steam..."
2,227300.0,True,1,"['Simulation', 'Driving', 'Open World', 'Reali...",Euro Truck Simulator 2,227300,"['Single-player', 'Steam Achievements', 'Steam..."
3,239030.0,True,1,"['Indie', 'Political', 'Simulation', 'Point & ...","Papers, Please",239030,"['Single-player', 'Steam Achievements', 'Steam..."
4,248820.0,True,2,"['Rogue-like', 'Indie', 'Platformer', 'Action'...",Risk of Rain,248820,"['Single-player', 'Multi-player', 'Co-op', 'Sh..."
...,...,...,...,...,...,...,...
2796,306040.0,True,2,"['Indie', 'Casual', 'Steampunk', 'Simulation',...",The Howler,306040,"['Single-player', 'Steam Trading Cards']"
2797,307130.0,True,1,"['Indie', 'Adventure', 'Action', 'Platformer',...",Asteria,307130,"['Single-player', 'Multi-player', 'Co-op', 'St..."
2798,284950.0,True,1,"['Casual', 'Puzzle', 'Indie', 'Simulation', 'S...",Pixel Puzzles: Japan,284950,"['Single-player', 'Steam Achievements', 'Steam..."
2799,209120.0,True,1,"['Fighting', 'Action', 'Arcade', '2D Fighter',...",Street Fighter X Tekken,209120,"['Single-player', 'Multi-player', 'Co-op', 'Sh..."


In [29]:
df_joinml1.specs=df_joinml1.specs.apply(literal_eval)## Setting correct datatype in column genres
df_joinml1.tags=df_joinml1.tags.apply(literal_eval)## Setting correct datatype in column genres

In [30]:
df_joinml1["genres"]=df_joinml1["specs"]
df_joinml1["genres1"]=df_joinml1["specs"]
df_joinml1["features"]=df_joinml1["specs"]

In [32]:
for i in range(df_joinml1.shape[0]):
    df_joinml1.genres.iloc[i]=" ".join(df_joinml1.tags.iloc[i])## Joining elements from genres list
for i in range(df_joinml1.shape[0]):
    df_joinml1.genres1.iloc[i]=" ".join(df_joinml1.specs.iloc[i])## Joining elements from genres list
for i in range(df_joinml1.shape[0]):
    df_joinml1.features.iloc[i]=df_joinml1.genres.iloc[i]+" "+df_joinml1.genres1.iloc[i]+" "+df_joinml1["app_name"].iloc[i] ## Joining elements from genres list and overview
df_joinml1["features"]=df_joinml1["features"].str.lower()

C:\Users\angeo\AppData\Local\Temp\ipykernel_21612\1269678798.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joinml1.genres.iloc[i]=" ".join(df_joinml1.tags.iloc[i])## Joining elements from genres list
C:\Users\angeo\AppData\Local\Temp\ipykernel_21612\1269678798.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joinml1.genres.iloc[i]=" ".join(df_joinml1.tags.iloc[i])## Joining elements from genres list
C:\Users\angeo\AppData\Local\Temp\ipykernel_21612\1269678798.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

In [33]:
df_MLF1=df_joinml1[["item_id","app_name","features"]] ### Extract columns which will be used
df_MLF1.reset_index(inplace=True)
df_MLF1.to_parquet("Dataset/Function_Dataset/MLF1.parquet")## Export dataset for ML function

In [34]:
df_MLF1## view data

,item_id,app_name,features
0,1250.0,Killing Floor,fps zombies co-op survival action multiplayer ...
1,22200.0,Zeno Clash,action indie surreal beat 'em up fps fighting ...
2,227300.0,Euro Truck Simulator 2,simulation driving open world realistic relaxi...
3,239030.0,"Papers, Please",indie political simulation point & click singl...
4,248820.0,Risk of Rain,rogue-like indie platformer action pixel graph...
...,...,...,...
2796,306040.0,The Howler,indie casual steampunk simulation strategy puz...
2797,307130.0,Asteria,indie adventure action platformer sandbox craf...
2798,284950.0,Pixel Puzzles: Japan,casual puzzle indie simulation singleplayer si...
2799,209120.0,Street Fighter X Tekken,fighting action arcade 2d fighter multiplayer ...


In [122]:
indices = df_MLF1[["item_id", "index"]]## Getting a dataset for finding new index

In [124]:
tfidf=TfidfVectorizer(stop_words="english",max_features=10000)## Setting tf-idf vector
tfidf_matrix=tfidf.fit_transform(df_MLF1["features"])## Setting tf-idf vectorizer with data
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)## Training model with the given data

In [125]:
def recomendacion(id):  # API function ML
    
    idx = indices[indices["item_id"]==id]## filter dataset with the parameter
    if idx.empty== True:## condition if dataset is empty
        recommendations=["No data available"]
    else:
        idy = idx["index"].iloc[0]## Search index
        sim_score = list(enumerate(cosine_sim[idy]))## Setting similarity
        sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)## Sorting results by score
        sim_score = sim_score[1:6]## Getting top score 5 movies
        movies_index = [i[0] for i in sim_score] ## Finding names
        recommendations = list(df_MLF1['app_name'].iloc[movies_index])## Making the list
    return {'titulo':id,'lista recomendada': recommendations}

In [132]:
recomendacion(10)## Testing ML function

{'titulo': 10,
 'lista recomendada': ['Counter-Strike: Condition Zero',
  'Counter-Strike: Source',
  'Counter-Strike: Global Offensive',
  'Insurgency',
  'Team Fortress Classic']}